In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importamos las librerías que necesitaremos

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Cargamos el data frame que vamos a estar trabajando

In [ ]:
df_original = pd.read_csv("../input/covid-world-vaccination-progress/country_vaccinations.csv", parse_dates=["date"])
df_original.head()

In [ ]:
df_original.info()

Notamos que el país y la fecha no tienen entradas faltantes, y son ideales para usarlos como identificador de nuestra entrada. Así que procedemos a ponerlos como índice. Adicionalmente, para el estudio que realizaremos, asumiremos que los datos son correctos (siempre que no haya evidencia de lo contrario), así que para el análisis, nos desharemos de las últimas dos columnas. Por último, notamos que hay muchos datos faltantes en algunas columnas y muchos datos que en teoría dependen de otros datos, así que deberíamos realizar algunas validaciones. 


In [ ]:
df = df_original.set_index(["country", "date"]).drop(["source_name","source_website", "iso_code"], axis=1)
df.head()

### Validaciones

In [ ]:
#total_vaccinations es el número de vacunas administradas, 
#mientras que people_vaccinated es el número total de personas vacunadas 
#(estos números pueden diferir porque hay )
cond = df["total_vaccinations"] == df["people_vaccinated"]
display(df[~cond & ~df["total_vaccinations"].isna()].head())

cond = df["total_vaccinations"] < df["people_vaccinated"]
display(df[cond].head()) #cambiar por un assert #revisar

Continuaremos viendo cuantos son los países y cuántos/cuales son los tipos de vacunas en el data frame. 

In [ ]:
countries = list(df.index.get_level_values(0).unique())
print(f"Hay {len(countries)} paises distintos")

In [ ]:
vaccines = list(df["vaccines"].unique())
vaccines[0:5]

Notemos que cada país puede tener más de una vacuna, por ejemplo 
$\texttt{vacines[3]}$: 


In [ ]:
vaccines[3]

Las vacunas están divididas por comas, así que las contaremos de otra forma

In [ ]:
vaccines1 = []
for lis in pd.Series(vaccines).str.split(","):
    vaccines1.extend(lis)

In [ ]:
vaccines = pd.Series(vaccines1).str.strip().unique()
print(f"Hay {len(vaccines)} vacunas diferentes:")
print(vaccines)

## Análisis exploratorio de datos(EDA)

Para comenzar, graficaremos el total de vacunaciones ($\texttt{total\-vaccinations}$) que ha habido en el tiempo por país. Para esto rellenaremos los valores faltantes con el valor inmediato anterior (forward fill). También hay que considerar que si hacemos una gráfica para cada país, sobrepoblaríamos la gráfica, entonces tomaremos un sample de 20 países. 



In [ ]:
cols_to_fill = ["total_vaccinations", "people_vaccinated", "total_vaccinations_per_hundred"] #falta escoger columnas #revisar 
df.loc[:, cols_to_fill].fillna(method="ffill", inplace=True)


In [ ]:
countries_to_plot = pd.Series(countries).sample(20)
samples_bool = df.index.get_level_values("country").isin(countries_to_plot)
df_to_plot = df[samples_bool]

In [ ]:
x = sns.relplot(data=df_to_plot, x="date", y="total_vaccinations", hue="country", style="country")
plt.title("20 Vacunación de una muestra de 20 paises");
#rotar las etiquetas del eje y #revisar

In [ ]:
x = sns.relplot(data=df_to_plot, x="date", y="total_vaccinations_per_hundred", hue="country", style="country")
#averiguar como poner ambas gráficas en una misma

Veremos cómo está méxico respecto a otros paises al empezar a vacunar. Comenzaremos analizando las primeras fechas de vacunación. 

In [ ]:
first_dates_per_country = df_original.groupby("country")["date"].min()
last_dates_per_country = df_original.groupby("country")["date"].max()
mex_first_date = first_dates_per_country["Mexico"]
str_mex_first_date = mex_first_date.strftime("%Y-%m-%d")
before_mex = len(first_dates_per_country[first_dates_per_country < first_dates_per_country["Mexico"]])
after_mex =len(first_dates_per_country[first_dates_per_country >= first_dates_per_country["Mexico"]])
print(f"México empezó a administrar vacunas en la fecha {str_mex_first_date} (yyyy-mm-dd).")
print(f"{before_mex} paises empezaron a vacunar antes de méxico y {after_mex} paises empezaron a vacunar después de méxico.")

In [ ]:
time_per_country = (last_dates_per_country - first_dates_per_country)

total_vaccines  =  df_original.groupby("country")["total_vaccinations"].max()
avg_total_vaccines_per_day = total_vaccines/(time_per_country.dt.days + 1)

total_vaccines_per_hundred = df_original.groupby("country")["total_vaccinations_per_hundred"].max()
avg_total_vaccines_hundred_per_day = total_vaccines_per_hundred/(time_per_country.dt.days + 1)

Ahora, realizaremos un pequeño análisis acerca de los promedios diarios de vacunación, tanto en términos absoultos como relativos (por cada 100 personas).

In [ ]:
avg_total_vaccines_per_day.sort_values(ascending=False).head(10).plot(kind="bar")
plt.xlabel("Pais")
plt.ylabel("Vacunas por día")
plt.title("10 Paises que ha vacunado a más personas por día")
plt.show()

In [ ]:
avg_total_vaccines_hundred_per_day.sort_values(ascending=False).head(10).plot(kind="bar")
plt.xlabel("Pais")
plt.ylabel("Promedio de vacunas por cada 100 personas diarias")
plt.title("10 Paises que ha vacunado a más personas por cada 100 por día")
plt.show()

In [ ]:
avg_total_vaccines_per_day.sort_values().head(10).plot(kind="bar")
plt.xlabel("Pais")
plt.ylabel("Vacunas por día")
plt.title("10 Paises que ha vacunado a menos personas por día")
plt.show()

In [ ]:
data = avg_total_vaccines_hundred_per_day.sort_values().head(10).plot(kind="bar")
plt.xlabel("Pais")
plt.ylabel("Promedio de vacunas por cada 100 personas diarias")
plt.title("10 Paises que ha vacunado a menos personas por cada 100 por día")
plt.show()

In [ ]:
avg_total_vaccines_per_day_mex = avg_total_vaccines_per_day["Mexico"]
avg_total_vaccines_hundred_per_day_mex = avg_total_vaccines_hundred_per_day["Mexico"]
print(f"México administra {avg_total_vaccines_per_day_mex} en promedio por día")
print(f"México administra {avg_total_vaccines_hundred_per_day_mex} en promedio por día por cada 100 personas")

cond1 = avg_total_vaccines_per_day.sort_values(ascending=False) > avg_total_vaccines_per_day_mex
num1 = len(avg_total_vaccines_per_day[cond1])
print(f"En términos absoultos, México se encunetra en el lugar número {num1}/{len(countries)}")

cond2 = avg_total_vaccines_hundred_per_day.sort_values(ascending=False) > avg_total_vaccines_hundred_per_day_mex
num2 = len(avg_total_vaccines_hundred_per_day[cond2])
print(f"En términos relativos, México se encuentra en el lugar número {num2}/{len(countries)}")

Por último, veremos qué vacunas han tenido más alcance al rededor del mundo. 

In [ ]:
vacc_dict = {}
for vacc in vaccines:
    grouped = df["vaccines"].groupby("country").sum()
    vacc_dict[vacc] = len(grouped[grouped.str.contains(vacc)])

In [ ]:
df_vacc = pd.DataFrame(list(vacc_dict.items()),columns = ['Vacuna', 'Cantidad'])
df_vacc

In [ ]:
sns.catplot(data=df_vacc.sort_values("Cantidad", ascending=False), x ="Cantidad", y="Vacuna", kind="bar")
plt.title("Cantidad de paises en los que se ha administrado esa vacuna")
plt.xlabel("Cantidad de paises")
plt.show()

La vacuna Pfizer/BioNTech es la que se ha administrado en más paises del mundo (74 países) y empatadas en último lugar, las vacunas que se han administrado en menos lugares del mundo han sido: Covaxin, EpiVacCorona y Johnson&Johnson. 